In [116]:
import requests
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup


pd.set_option('display.max_columns', 70)


In [117]:
def get_df_from_url(url):
    # Send a GET request to the website
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    response.raise_for_status()  # Ensure the request was successful

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table in the page (you might need to adjust the class or ID if the table is dynamic)
    table = soup.find("table")

    # Extract the headers
    headers = [th.get_text(strip=True) for th in table.find_all("th")]

    # Extract the rows
    rows = []
    for tr in table.find_all("tr")[1:]:  # Skip the header row
        cells = [td.get_text(strip=True) for td in tr.find_all("td")]
        rows.append(cells)


    df = pd.DataFrame(rows, columns=headers)

    return df

In [118]:
# Define the URL
base_url = "https://www.ft.com/ft1000-"
years = [str(num) for num in range(2019, 2025)]

years


['2019', '2020', '2021', '2022', '2023', '2024']

In [119]:

column_names = []
df_list = []

for year in years:
    url = base_url + year

    df = get_df_from_url(url)


    column_name = df.columns.tolist()
    print("Column names:", column_name)


    column_names.append(column_name)
    df_list.append(df)

Column names: ['Rank', 'Company', 'In 2018 list?', 'In 2017 list?', 'Country', 'Sector', 'Absolute revenue growth', 'Revenue CAGR 2014-17', 'Revenue 2017 (€m)', 'Employee growth 2014-17', 'Employees 2017', 'Founded']
Column names: ['Rank', 'Name', 'in 2019 ranking', 'in 2018 ranking', 'Country', 'FT Category', 'Absolute Growth Rate [in %]', 'Compound Annual   Growth Rate (CAGR) [in %]', 'Revenue 2018 [in €m]', 'Revenue 2015 [in €m]', 'Number of   employees 2018', 'Founding Year']
Column names: ['Rank', 'Name', 'in 2020 ranking', 'in 2019 ranking', 'Country', 'FT Category', 'Absolute Growth Rate %', 'Compound Annual Growth Rate (CAGR) %', 'Revenue 2019 €', 'Revenue 2016 €', 'Number of employees 2019', 'Number of employees 2016', 'Founding Year']
Column names: ['Rank', 'Name', 'in 2021 ranking', 'in 2020 ranking', 'Country', 'Sector', 'Absolute Growth Rate %', 'Compound Annual Growth Rate (CAGR) %', 'Revenue 2020 (€)', 'Revenue 2017 (€)', 'Number of employees 2020', 'Number of employees 

In [120]:
cols_to_delete = [
    'In 2018 list?', 
    'In 2017 list?',
    'in 2019 ranking',
    'in 2018 ranking',
    'in 2020 ranking',
    'in 2021 ranking',
    'in 2022 ranking',
    'Employee growth 2014-17',
]

company_names = []
df_list_copy = []



def rename_columns(df, col_names, new_name):
    check = 0
    for col in col_names:
        if check == 1:
            break
        if col in df.columns:
            df.rename(columns={col: new_name}, inplace=True)
            check = 1
    return df

def clean_dataframe_column_names(df):
    """
    Cleans the column names of a DataFrame by:
    - Stripping extra spaces (ensuring a single space between words).
    - Capitalizing the first word of each column name.

    Parameters:
        df (pd.DataFrame): The DataFrame whose column names need cleaning.

    Returns:
        pd.DataFrame: The DataFrame with cleaned column names.
    """
    # Clean each column name
    cleaned_columns = [" ".join(col.split()).title() for col in df.columns]
    # Rename columns in the DataFrame
    df.columns = cleaned_columns
    return df



for df_index, df_val in enumerate(df_list):

    df = df_val.copy(deep=True)


    # print('-'*80)
    # print(df.columns)
    ##################################################
    # rename comapny name
    ##################################################
    col_names = [
        'Company',
        ]
    df = rename_columns(df, col_names, 'Name')


    ##################################################
    # rename revenue growth
    ##################################################
    col_names = [
        'Absolute Growth Rate [in %]',
        'Absolute Growth Rate %',
        'Absolute Growth Rate (%)',
        'Absolute revenue growth',
        ]
    df = rename_columns(df, col_names, 'Absolute Growth Rate')


    ##################################################
    # rename Compound Annual Growth Rate
    ##################################################
    col_names = [
        'Compound Annual Growth Rate (CAGR) %',
        'Compound Annual Growth Rate (%)',
        'Revenue CAGR 2014-17',
        'Compound Annual   Growth Rate (CAGR) [in %]'
        'CAGR'
        ]
    df = rename_columns(df, col_names, 'Compound Annual Growth Rate')


    ##################################################
    # rename sector
    ##################################################
    col_names = [
        'Sector',
        'FT Category',
        ]
    df = rename_columns(df, col_names, 'Sector')


    ##################################################
    # rename founding year
    ##################################################
    col_names = [
        'Founding Year',
        'Founded',
        ]
    df = rename_columns(df, col_names, 'Founded')



    ##################################################
    # Rename columns containing 'employees'
    ##################################################
    df.columns = [
        re.sub(r'(?:[ ]*[Nn]umber[ ]+of[ ]+)?[Ee]mployees[ ]+(\d{4})', r'Employees \1', col, flags=re.IGNORECASE) 
        if 'employees' in col.lower() else col
        for col in df.columns
    ]

    ##################################################
    # Rename columns containing 'revenue'
    ##################################################
    df.columns = [
        re.sub(r'[\[\(].*?[\]\)]|€|[€]', '', col).strip()  # Clean and preserve spaces
        for col in df.columns
    ]

    # print(df.columns)
    
    # print(df.columns)
    df_list_copy.append(df)
    # print(df_list_copy[df_index].columns)
    # print('-'*80)



unique_col_names = []


for df_index, df_val in enumerate(df_list_copy):

    df = df_val.copy()

    ##################################################
    # Delete columns
    ##################################################
    df = df.drop(columns=[col for col in cols_to_delete if col in df.columns])

    df = clean_dataframe_column_names(df)

    # print(df.columns)
    df_list_copy[df_index] = df

    for name in df.columns:
        if name not in unique_col_names:
            unique_col_names.append(name)

# print(unique_col_names)





In [144]:
mega_df_list = []

for df_index, df_val in enumerate(df_list_copy):
    df = df_val.copy()
    for col in unique_col_names:
        if col not in df.columns:
            df[col] = np.nan
    
    df['Year'] = years[df_index]

    mega_df_list.append(df)



master_df = pd.concat(mega_df_list, ignore_index=True)


## Clean up the df

In [155]:
master_df['Sector'] = master_df['Sector'].apply(lambda x: " ".join(str(x).split()).title() if pd.notna(x) else x)
master_df['Country'] = master_df['Country'].apply(lambda x: " ".join(str(x).split()).title() if pd.notna(x) else x)



In [146]:
master_df.head()

,Rank,Name,Country,Sector,Absolute Growth Rate,Compound Annual Growth Rate,Revenue 2017,Employees 2017,Founded,Revenue 2018,Revenue 2015,Employees 2018,Revenue 2019,Revenue 2016,Employees 2019,Employees 2016,Revenue 2020,Employees 2020,Revenue 2021,Employees 2021,Revenue 2022,Employees 2022,Year
0,1,Blue Motor Finance,United Kingdom,Financial Services,"51,364%",701.4%,61.4,155,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1,2,Deliveroo,United Kingdom,Food & Beverage,"15,749%",441.2%,316.1,1664,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
2,3,Taxify,Estonia,Technology,"12,231%",397.7%,17.8,350,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
3,4,Solectric,Germany,Technology,"7,772%",328.6%,40.9,30,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
4,5,Psioxus Therapeutics,United Kingdom,Pharmaceuticals,"7,126%",316.5%,64.4,69,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019


In [165]:
sector_mapping = {
    'Aerospace & Defence':'Defence',
    'Advertising & Marketing': 'Sales & Marketing',
    'Advertising': 'Sales & Marketing',
    'Sales & Marketing': 'Sales & Marketing',
    'Sales And Marketing': 'Sales & Marketing',
    'Automobiles': 'Automotive', 
    'Agricultural Commodities': 'Agricultural',
    'Agriculture, Forestry & Fishing': 'Agricultural',
    'Apparel & Fashion': 'Fashion',
    'Chemicals & Pharmaceuticals': 'Chemicals',
    'Construction & Engineering': 'Construction',
    'Education & Social Services': 'Education',
    'Energy & Utilities': 'Energy',
    'Food & Beverages': 'Food & Beverage',
    'Fintech, Financial Services & Insurance': 'Financial Services',
    'Fintech, Financial Services & Insurance': 'Financial Services',
    'Fintech': 'Financial Services',
    'Healh': 'Health',
    'Health Care & Life Sciences': 'Health',
    'Law': 'Legal', 
    'Legal & Accounting Services': 'Legal',
    'Media & Telecommunications': 'Media',
    'Telecoms': 'Media',
    'Pharmaceuticals & Cosmetics':'Pharmaceuticals',
    'Precious Metals':'Mining',
    'Property':'Real Estate',
    'Restaurants':'Hospitality',
    'Hospitality & Travel':'Hospitality',
    'Logistics & Transportation':'Transport',
    'Waste Management & Recycling':'Waste Management'
}


for key, val in sector_mapping.items():
    master_df.loc[master_df["Sector"] == key, "Sector"] = val




country_mapping = {
    'Uk':'United Kingdom',
    'UK':'United Kingdom',
    'The Netherlands': 'Netherlands',
}

for key, val in country_mapping.items():
    master_df.loc[master_df["Country"] == key, "Country"] = val



master_df.head()

,Rank,Name,Country,Sector,Absolute Growth Rate,Compound Annual Growth Rate,Revenue 2017,Employees 2017,Founded,Revenue 2018,Revenue 2015,Employees 2018,Revenue 2019,Revenue 2016,Employees 2019,Employees 2016,Revenue 2020,Employees 2020,Revenue 2021,Employees 2021,Revenue 2022,Employees 2022,Year
0,1,Blue Motor Finance,United Kingdom,Financial Services,"51,364",701.4,61.4,155,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1,2,Deliveroo,United Kingdom,Food & Beverage,"15,749",441.2,316.1,1664,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
2,3,Taxify,Estonia,Technology,"12,231",397.7,17.8,350,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
3,4,Solectric,Germany,Technology,"7,772",328.6,40.9,30,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
4,5,Psioxus Therapeutics,United Kingdom,Pharmaceuticals,"7,126",316.5,64.4,69,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019


In [172]:
master_df.describe()

,Rank,Name,Country,Sector,Absolute Growth Rate,Compound Annual Growth Rate,Revenue 2017,Employees 2017,Founded,Revenue 2018,Revenue 2015,Employees 2018,Revenue 2019,Revenue 2016,Employees 2019,Employees 2016,Revenue 2020,Employees 2020,Revenue 2021,Employees 2021,Revenue 2022,Employees 2022,Year
count,6001,6000,6000,6000,6000,6000,2000,2000,6000,2000,1000,2000,2000,1000,2000,1000,1000,1000,1000,1000,1000,1000,6001
unique,1001,4703,34,46,4513,2301,1294,232,69,1730,438,270,1972,992,263,129,999,226,1000,257,999,245,6
top,530,Eco CO2,Italy,Technology,162,39.2,1.8,2,2012,2.04,0.56,3,"3,200,000","2,100,000",3,5,"1,900,000",5,"83,062,822",10,"3,700,000",15,2021
freq,9,6,1465,766,9,27,30,96,682,5,12,89,3,3,87,67,2,32,1,33,2,34,1001


In [176]:
master_df['Absolute Growth Rate'] = master_df['Absolute Growth Rate'].str.strip().str.lower().str.replace("%", "").str.replace(",", "")
master_df['Compound Annual Growth Rate'] = master_df['Compound Annual Growth Rate'].str.strip().str.lower().str.replace("%", "").str.replace(",", "")


master_df['Absolute Growth Rate'] = pd.to_numeric(master_df['Absolute Growth Rate'],errors = 'coerce')
master_df['Compound Annual Growth Rate'] = pd.to_numeric(master_df['Compound Annual Growth Rate'],errors = 'coerce')


# master_df['Absolute Growth Rate'] = df['Absolute Growth Rate'].astype(float)
# master_df['Compound Annual Growth Rate'] = df['Compound Annual Growth Rate'].astype(float)

master_df.head()

,Rank,Name,Country,Sector,Absolute Growth Rate,Compound Annual Growth Rate,Revenue 2017,Employees 2017,Founded,Revenue 2018,Revenue 2015,Employees 2018,Revenue 2019,Revenue 2016,Employees 2019,Employees 2016,Revenue 2020,Employees 2020,Revenue 2021,Employees 2021,Revenue 2022,Employees 2022,Year
0,1,Blue Motor Finance,United Kingdom,Financial Services,51364.0,701.4,61.4,155,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
1,2,Deliveroo,United Kingdom,Food & Beverage,15749.0,441.2,316.1,1664,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
2,3,Taxify,Estonia,Technology,12231.0,397.7,17.8,350,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
3,4,Solectric,Germany,Technology,7772.0,328.6,40.9,30,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019
4,5,Psioxus Therapeutics,United Kingdom,Pharmaceuticals,7126.0,316.5,64.4,69,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019


In [177]:
current_directory = os.getcwd()

# Define the file path for saving the CSV
file_path = os.path.join(current_directory, 'data.csv')

# Save the DataFrame to a CSV file
master_df.to_csv(file_path, index=False)


# Analysis

In [148]:
# get top 10 most frequent names
n = 20
frequenct_sector = master_df['Sector'].value_counts()[:n].index.tolist()

frequenct_sector

['Technology',
 'Construction',
 'Ecommerce',
 'It & Software',
 'Financial Services',
 'Sales & Marketing',
 'Support Services',
 'Retail',
 'Energy',
 'Management Consulting',
 'Transport',
 'Industrial Goods',
 'Food & Beverage',
 'Media',
 'Automotive',
 'Health',
 'Real Estate',
 'Wholesale',
 'Travel & Leisure',
 'Education']

## Run to load info from .csv

In [82]:
# df = pd.read_csv('data/web-scraped-result.csv',
#                 delimiter=',',            # Comma as delimiter
#                  quotechar='"',            # Use double quotes for quoting
#                 quoting=csv.QUOTE_MINIMAL,  # Minimal quoting (or csv.QUOTE_ALL as needed)
#                 doublequote=True, 
#                 encoding='utf-8',
#                 )
# df.head()